# 01 - Feast Feature Store Setup

![Workflow](../docs/01-features-workflow.png)

**Raw Data → Feature Engineering → Feast Apply → Materialize**

**Prerequisites:** PostgreSQL, PVC, RayCluster running (see `manifests/00-04`).

In [ ]:
%pip install -q "feast[postgres]==0.59.0" pandas pyarrow psycopg2-binary
import os, shutil, subprocess
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta, timezone

## Configuration

In [ ]:
FEATURE_REPO = Path("/shared/feature_repo")
DATA_DIR = Path("/shared/data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

START_DATE, WEEKS, STORES, DEPTS, SEED = "2022-01-01", 104, 45, 14, 42
print(f"Records: {WEEKS * STORES * DEPTS:,}")

## Generate Sales Data

In [ ]:
np.random.seed(SEED)
base_date = datetime.fromisoformat(START_DATE).replace(tzinfo=timezone.utc)
HOLIDAYS = {6, 27, 36, 47, 51}

records = []
for week in range(WEEKS):
    dt = base_date + timedelta(weeks=week)
    woy, month = dt.isocalendar()[1], dt.month
    seasonal = 1 + 0.3 * np.sin(2 * np.pi * woy / 52)
    for s in range(1, STORES + 1):
        for d in range(1, DEPTS + 1):
            sales = max(0, (50000 + s*5000) * (0.5 + d*0.2) * seasonal * (1.5 if woy in HOLIDAYS else 1) + np.random.normal(0, 2000))
            records.append({"store_id": s, "dept_id": d, "event_timestamp": dt, "weekly_sales": round(sales, 2),
                "week_of_year": woy, "month": month, "quarter": (month-1)//3+1, "is_holiday": int(woy in HOLIDAYS),
                "temperature": round(60 + 20*np.sin(2*np.pi*woy/52) + np.random.normal(0,5), 1),
                "fuel_price": round(3 + 0.5*np.random.random(), 2), "cpi": round(220 + week*0.1, 1), "unemployment": round(5 + np.random.normal(0, 0.5), 1)})

sales_df = pd.DataFrame(records).sort_values(["store_id", "dept_id", "event_timestamp"]).reset_index(drop=True)
print(f"Generated {len(sales_df):,} rows")

In [ ]:
# Lag + rolling features
for lag in [1, 2, 4, 8]:
    sales_df[f"lag_{lag}"] = sales_df.groupby(["store_id", "dept_id"])["weekly_sales"].shift(lag)

g = sales_df.groupby(["store_id", "dept_id"])["weekly_sales"]
sales_df["rolling_mean_4w"] = g.transform(lambda x: x.rolling(4, min_periods=1).mean())
sales_df["rolling_std_4w"] = g.transform(lambda x: x.rolling(4, min_periods=2).std()).fillna(0)
sales_df["sales_vs_avg"] = (sales_df["weekly_sales"] / sales_df["rolling_mean_4w"].replace(0, 1)).fillna(1)

for lag in [1, 2, 4, 8]:
    sales_df[f"lag_{lag}"] = sales_df[f"lag_{lag}"].fillna(sales_df["rolling_mean_4w"])
sales_df = sales_df.fillna(0)
print(f"Features: {len(sales_df.columns)}")

In [ ]:
# Save
sales_df.to_parquet(DATA_DIR / "sales_features.parquet", index=False)
stores = pd.DataFrame([{"store_id": s, "dept_id": d, "event_timestamp": base_date, "store_type": ["A","B","C"][s%3], "store_size": 100000+s*10000, "region": f"region_{(s-1)//15+1}"} for s in range(1, STORES+1) for d in range(1, DEPTS+1)])
stores.to_parquet(DATA_DIR / "store_features.parquet", index=False)
print(f"✅ Saved to {DATA_DIR}")

## Setup Feast

In [ ]:
FEATURE_REPO.mkdir(parents=True, exist_ok=True)
src_dir = Path("../feature_repo")
for f in ["feature_store.yaml", "feature_store_ray.yaml", "features.py"]:
    src = src_dir / f
    if src.exists():
        shutil.copy(src, FEATURE_REPO / f)
        print(f"✅ {f}")
    else:
        print(f"❌ Missing: {src}")

## Feast Apply

In [ ]:
os.chdir(str(FEATURE_REPO))
result = subprocess.run(["feast", "apply"], capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print(f"ERROR: {result.stderr}")

## Feast Materialize

In [ ]:
end_ts = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S")
result = subprocess.run(["feast", "materialize", f"{START_DATE}T00:00:00", end_ts], capture_output=True, text=True, cwd=str(FEATURE_REPO))
print(result.stdout)
if result.returncode != 0:
    print(f"ERROR: {result.stderr}")

## Verify

In [ ]:
from feast import FeatureStore
store = FeatureStore(repo_path=str(FEATURE_REPO))
print(f"Views: {[fv.name for fv in store.list_feature_views()]}")
print(f"Services: {[fs.name for fs in store.list_feature_services()]}")

features = store.get_online_features(features=["sales_features:weekly_sales", "sales_features:lag_1"], entity_rows=[{"store_id": 1, "dept_id": 1}]).to_dict()
print(f"\n✅ Online: {features}")

**Next:** `02-training.ipynb`